# モンテカルロ法

乱数を用いた試行を繰り返すことによって確率的に近似解を求める手法を総称してモンテカルロ法（Monte Carlo method）と呼ぶ。ここではモンテカルロ法を用いた数値実験（すなわちモンテカルロ・シミュレーション）によって、問題を解いてみよう。

## 今回のねらい

- モンテカルロ法によって問題を解く基本的な考え方を習得する。
- モンテカルロ法で得られる結果には統計的な誤差が付随することを理解する。

## コイン投げ

次のような確率を求める問題を考えてみよう。

> 表と裏が等確率でランダムに出るコインがある。このコインを3回投げたとき、コインの表が2回以上連続で出る確率を求めよ。

<!-- textlint-disable ja-technical-writing/max-ten -->

3回投げるので、$2^3=8$通りのコインの出方をすべて書き出して調べればこの確率を求めることができる。表を「お」、裏を「う」として表すと、8通りのうち、コインの表が2回以上連続で出るパターンは「おおお」、「おおう」、「うおお」の3通りである。よって求める確率は$3/8 = 0.375$となる。

<!-- textlint-enable -->

では、次の問題はどうだろうか。

> 前問と同じコインを**100回投げた**とき、コインの表が（少なくとも1度は）**6回以上連続**で出る確率を求めよ」

この問題を$2^{100} \approx 1.3 \times 10^{30}$通りのコインの出方をすべて書き出すことで解こうとする人はいないだろう。幸い、この確率は計算によって出すことができるのだが<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)、ここでは次のような実験を考える。

1. コインを100回投げる。表が6回以上連続で出たかを記録する。
2. これを多くの回数繰り返す。たとえば$n$回繰り返したとして、その中で表が6回以上連続で出たのが$m$回だったとすれば、大数の法則より求める確率は近似的に$m/n$である。近似は$n$を大きくするにつれて良くなる。

当然、人間がコインを何回も（たとえば$n=10^4$として合計100万回）投げるのは時間的および精神的に難しいだろう。しかし、コンピューター上での疑似乱数を用いた模擬実験（モンテカルロ・シミュレーション）ならばこれが可能である。

まず、コインを何回か投げて、その中である回数以上表が連続で出たかどうかを判定することを考えよう。ループと条件分岐を使って真面目に作ってもよいのだが、ここでは手抜きをする。[`random`モジュール](https://docs.python.org/ja/3/library/random.html)には[`choices`関数](https://docs.python.org/ja/3/library/random.html#random.choices)が用意されている。第1引数に文字列（あるいはリストなど）、`k`というオプション引数に数字を与えると、第1引数の要素をランダムに1つ選ぶことを$k$回繰り返して作ったリストを返す。

In [ ]:
import random

# 表は「お」、裏は「う」として、ランダムに10回投げたコインの目を表すリスト
random.choices("おう", k=10)

文字のリストを連結するには、文字列オブジェクトの[`join`メソッド](https://docs.python.org/ja/3/library/stdtypes.html#str.join)を使う。

In [ ]:
import random

# 表は「お」、裏は「う」として、ランダムに10回投げたコインの目を表す文字列
"".join(random.choices("おう", k=10))

この文字列から任意のパターン（たとえば「おおお」）があるかを調べるのは正規表現を用いれば容易である。ここでは[`re`モジュール](https://docs.python.org/ja/3/library/re.html)の[`search`関数](https://docs.python.org/ja/3/library/re.html#re.search)を使う<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)。

In [ ]:
import random
import re

# 10回コインを投げて、3回以上表が連続で出たかどうか調べる
sequence = "".join(random.choices("おう", k=10))
if re.search("おおお", sequence):
    print(f"{sequence} : あった")
else:
    print(f"{sequence} : なかった")

あとはこのような試行を実際にたくさん繰り返し、パターンが見つかった回数を数えて確率を求めればよい。

In [ ]:
%%time

import random
import re

n = 10**2  # サンプルサイズ

count = 0  # パターンがあった回数

for _ in range(n):
    # コインを100回投げて、その中で表が6回以上連続したパターンがあるか調べる
    sequence = "".join(random.choices("おう", k=100))
    if re.search("おおおおおお", sequence):
        count += 1

# 結果と統計誤差
result = count / n
error = result / n**0.5

print(f"試行回数: {n}")
print(f"あった回数: {count}")
print(f"結果: {result}")
print(f"誤差: {error}")

セルの最初に書いた`%%time`はJupyterのマジックコマンドで、セルの実行にかかった時間を表示してくれる。一度、上のセルを実行したのち、どのくらい計算時間が掛かるか予測しながらサンプルサイズを増やしてみよう。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/max-ten -->

乱数を用いた数値実験なので、結果は毎回少しずつ違うはずである。前回学んだ中心極限定理を用いて統計的な揺らぎによる誤差を考えると、試行回数（サンプルサイズ）$n$ に対して、結果に対する相対誤差は$1/\sqrt{n}$に比例する（ここでは比例係数を無視して単に$1/\sqrt{n}$を掛けたものを誤差の見積もりとしている）<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3)<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4)。よって、近似の精度を1桁上げたければ、サンプルサイズを100倍にする必要がある（当然、計算時間も100倍かかることになる）。モンテカルロ法を使った数値実験では、結果とともにその誤差の範囲を把握することが重要である<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5)。

<!-- textlint-enable -->

## 練習問題

<!-- textlint-disable ja-technical-writing/no-doubled-joshi -->

(1) コインを100回投げたとき、表でも裏でもよいので同じ面が6回以上連続して出る確率を求めよ<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6)。

<!-- textlint-enable -->

## 円周率

<!-- textlint-disable ja-technical-writing/ja-no-weak-phrase,ja-engineering-paper/prh -->

次に考えるのは円周率の値である。円周率の値を求めることのような決定論的問題が、どうしてモンテカルロ法と結びつくのかと疑問に思うかもしれない。ここでは、以下のような方法によって、円周率の近似値を確率的に計算する。

<!-- textlint-enable -->

![モンテカルロ法による円周率の計算](https://tueda.github.io/PS2022SS/notebooks/images/mc_pi.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図1. 半径1の四分円と外接する正方形

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-engineering-paper/use-si-units -->

図1 (a) のような、一辺の長さが1の正方形と、その左下隅を中心とする半径1の四分円を考える。この正方形の中で繰り返しランダムに点を発生させよう（図1 (b)）。するとランダムに発生させた点が四分円の中に入る確率は四分円の面積に比例することから、

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/ja-no-mixed-period -->

$$
\frac{\text{（四分円の面積）}}{\text{（正方形の面積）}}
\simeq
\frac{\text{（四分円の中に入った点の数）}}{\text{（正方形の中で発生させた点の数）}} ,
\tag{1}
$$

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/max-ten,jtf-style/4.3.1.丸かっこ（） -->

という関係が近似的に成り立つであろう。右辺については、たとえば、実際に正方形と四分円を紙に描き、それを壁に貼って何回もダーツをランダムに当てたりすれば実験的に測定できる<a name="cite_ref-7"></a>[<sup>[7]</sup>](#cite_note-7)。今回はこれをモンテカルロ・シミュレーションで行う。四分円の面積は$\pi/4$で、正方形の面積は$1$なので、式(1)の右辺の実験値に$4$を掛けると円周率$\pi$が求まる。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,jtf-style/4.3.7.山かっこ<>,ja-technical-writing/ja-no-mixed-period -->

正方形の中のランダムな点 $(x_i, y_i)$を1つ発生させるには、[random関数](https://docs.python.org/ja/3/library/random.html#random.random)を2回呼び出して$x_i$と$y_i$をそれぞれ発生させればよい（$0 \le x_i < 1$、$0 \le y_i < 1$のようになるよう発生させる）。この点が四分円の中にあるかどうかは、

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/ja-no-mixed-period -->

$$
\begin{cases}
x^2 + y^2 < 1 , & \text{円の内部} , \\
x^2 + y^2 > 1 , & \text{円の外部} ,
\end{cases}
\tag{2}
$$

<!-- textlint-enable -->

のようにして判断できる<a name="cite_ref-8"></a>[<sup>[8]</sup>](#cite_note-8)。この試行を繰り返し、点が円の内部に入った回数を数えておけば、式 (1) の右辺が数値実験によって求まる。

In [ ]:
%%time

import random

n = 10**2  # サンプルサイズ

count = 0  # 中に入った回数

for _ in range(n):
    # 正方形内にランダムな点を発生させて、円の1/4の中に入ったか調べる
    x = random.random()
    y = random.random()

    if x**2 + y**2 < 1:
        count += 1

# 結果と統計誤差
result = 4 * count / n
error = result / n**0.5

print(f"試行回数: {n}")
print(f"入った回数: {count}")
print(f"結果: {result}")
print(f"誤差: {error}")

## 練習問題

(2) 皆さんは円周率の値を知っている（`math.pi`）。上のプログラムで、サンプルサイズを変えながら何回か計算してみて、結果が誤差の見積りの範囲内で合っているのか確かめてみよう<a name="cite_ref-9"></a>[<sup>[9]</sup>](#cite_note-9)。

## 可視化する

せっかくなので、直感的理解を助けるために、上のモンテカロ法の数値実験を可視化してみよう（実用性はないが教育的ではあるだろう）。つまり、ランダムに発生させた点を散布図としてグラフにしてみる。円の内部に入った点は青、円の外部の点は赤で表すとしよう。次のプログラムでは散布図を[Axesオブジェクト](https://matplotlib.org/api/axes_api.html#the-axes-class)の[scatterメソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.scatter.html#matplotlib-axes-axes-scatter)を使って描画し、また、[add_patchメソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.add_patch.html#matplotlib.axes.Axes.add_patch)を使って円を描画している。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 100  # サンプルサイズ（グラフを描くので少なめに）

# グラフ用のデータ
xpoints = []
ypoints = []
colors = []

count = 0  # 中に入った回数

for _ in range(n):
    # 正方形内にランダムな点を発生させて、円の1/4の中に入ったか調べる
    x = random.random()
    y = random.random()

    if x**2 + y**2 < 1:
        count += 1
        c = "blue"  # 内部は青い点
    else:
        c = "red"  # 外部は赤い点

    # グラフ用
    xpoints.append(x)
    ypoints.append(y)
    colors.append(c)

# 結果と統計誤差
result = 4 * count / n
error = result / n**0.5

print(f"試行回数: {n}")
print(f"入った回数: {count}")
print(f"結果: {result}")
print(f"誤差: {error}")

# グラフを作る
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=5, c=colors)  # size と color を指定
ax.add_patch(plt.Circle((0, 0), radius=1, fc="none", ec="green", linewidth=2))  # 円を描く
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect("equal")
ax.grid()
plt.show()

## 球の体積

![三次元球](https://tueda.github.io/PS2022SS/notebooks/images/sphere.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図2. 半径1の球

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,jtf-style/4.3.7.山かっこ<> -->

上では円周率の近似値を得るために、正方形と四分円の面積の比をモンテカルロ法により求めていた。同様にして、半径1の球の体積を求めてみよう。ランダムな点を一辺の長さが1の立方体（$0 \le x < 1$、$0 \le y < 1$、$0 \le z < 1$）に発生させ、原点からの距離の2乗が1より小さいかどうかで球の内部か外部を判定する。立方体に入っている球の部分は全体の$1/8$であるから、得られた結果に$8$を掛ければ球全体の体積が求まる。

<!-- textlint-enable -->

In [ ]:
%%time

import random

n = 10**6  # サンプルサイズ

count = 0  # 中に入った回数

for _ in range(n):
    # 立方体内にランダムな点を発生させて、球の1/8の中に入ったか調べる
    x = random.random()
    y = random.random()
    z = random.random()

    if x**2 + y**2 + z**2 < 1:
        count += 1

# 結果と統計誤差
result = 8 * count / n
error = result / n**0.5

print(f"試行回数: {n}")
print(f"入った回数: {count}")
print(f"結果: {result}")
print(f"誤差: {error}")

## 4次元超球の超体積

2次元平面で単位円の面積、3次元空間で単位球の体積を考えたように、4次元空間で半径が$1$である4次元超球の超体積（つまり中心からの距離が$1$以内である点の集合が占める領域）を考える。この超体積をモンテカルロ法で求めてみよう<a name="cite_ref-10"></a>[<sup>[10]</sup>](#cite_note-10)。この場合、相対誤差の見積もりを$1 / \sqrt{n}$とすると少々甘くなるので、代わりに次の式を使って評価する。

<!-- ja-technical-writing/ja-no-mixed-period,textlint-disable jtf-style/4.3.1.丸かっこ（） -->

$$
\text{(相対誤差)}
= \sqrt{\frac{1-p}{n p}}
\simeq \sqrt{\frac{1-\bar{p}}{n \bar{p}}}.
\tag{3}
$$

<!-- textlint-enable -->

ここで$p$はランダムに発生させた点が4次元超球に入る真の確率であるが、我々はその値を知らない（モンテカルロ法で近似的に求めようとしている）。真の確率$p$の代わりに、数値実験においてランダムに発生させた点が4次元超球に入った割合$\bar{p}$（$=$ `count / n`）で近似的に評価する。

In [ ]:
%%time

import random

n = 10**6  # サンプルサイズ

count = 0  # 中に入った回数

for _ in range(n):
    # 4次元超立方体内にランダムな点を発生させて、4次元超球の1/16の中に入ったか調べる
    x = random.random()
    y = random.random()
    z = random.random()
    w = random.random()

    if x**2 + y**2 + z**2 + w**2 < 1:
        count += 1

# 結果と統計誤差
p = count / n
result = 16 * p
error = result * ((1 - p) / (n * p)) ** 0.5

print(f"試行回数: {n}")
print(f"入った回数: {count}")
print(f"結果: {result}")
print(f"誤差: {error}")

## 練習問題

(3) 同様に5次元超球、6次元超球（あるいはさらに高次元での超球）の超体積を考えてみよう<a name="cite_ref-11"></a>[<sup>[11]</sup>](#cite_note-11)。

## 演習課題

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/prh -->

(1) 曲線 $y = \sqrt{x}$ の $0 \le x \le 1$ の部分と$x=1$の直線、および$x$軸に囲まれた領域の面積（つまり$\int_0^1 \sqrt{x} \, dx$）をモンテカルロ法にて求め<a name="cite_ref-12"></a>[<sup>[12]</sup>](#cite_note-12)、結果を誤差の見積もりとともに表示せよ。相対誤差がおおよそ1%以下になるようにサンプルサイズを取れ。

<!-- textlint-enable -->

In [ ]:
# 課題解答8.1  <-- 提出する際に、この行を必ず含めること。

import random

(2) 第5回の課題(2)に出てきた内トロコイドの媒介変数表示において、$a=4/\sqrt{6}$、$b=c=1/\sqrt{6}$としたときの曲線（アステロイド；astroid）は陰関数表示で次のように書ける。

$$
\left(x^2 + y^2 - \frac{8}{3}\right)^3 + 72 x^2 y^2 = 0. \tag{4}
$$

![アステロイド](https://tueda.github.io/PS2022SS/notebooks/images/astroid.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図3. アステロイド

<!-- textlint-enable -->

<!-- textlint-disable ja-engineering-paper/prh -->

この曲線で囲まれた領域（図3）の面積<a name="cite_ref-13"></a>[<sup>[13]</sup>](#cite_note-13)をモンテカルロ法にて求め、結果を誤差の見積もりとともに表示せよ。相対誤差がおおよそ1%以下になるようにサンプルサイズを取れ。ただし、相対誤差の評価には式<!-- eqref -->(3)を用いること。また、乱数によって発生させる$x$と$y$の範囲に注意せよ<a name="cite_ref-14"></a>[<sup>[14]</sup>](#cite_note-14)。

<!-- textlint-enable -->

In [ ]:
# 課題解答8.2  <-- 提出する際に、この行を必ず含めること。

import random

## 発展課題

<!-- textlint-disable jtf-style/1.1.1.本文 -->

余裕があればやってみてください。

<!-- textlint-enable -->

(3) 円周率を10進法で表したとき、小数点以下762桁目から767桁目まで6個の9が並ぶ。実際、小数点以下701桁目から800桁目を書き出すと次のようになる。

<!-- textlint-disable ja-technical-writing/sentence-length -->

4201995611 2129021960 8640344181 5981362977 4771309960 5187072113 4**999999**837 2978049951 0597317328 1609631859

<!-- textlint-enable -->

<!-- textlint-disable ja-engineering-paper/prh,jtf-style/4.3.1.丸かっこ（） -->

この6個の9の並びを（真偽は別として）[リチャード・ファインマン](https://ja.wikipedia.org/wiki/%E3%83%AA%E3%83%81%E3%83%A3%E3%83%BC%E3%83%89%E3%83%BBP%E3%83%BB%E3%83%95%E3%82%A1%E3%82%A4%E3%83%B3%E3%83%9E%E3%83%B3)の逸話から[ファインマン・ポイント](https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%B3%E3%83%9E%E3%83%B3%E3%83%BB%E3%83%9D%E3%82%A4%E3%83%B3%E3%83%88)（Feynman point）と呼ぶことがある。円周率は10進小数表示において正規数（無限小数表示において数字がランダムに現れる実数）だと信じられている。もしそれが正しいとして、このような早い桁に6個の同じ数字の並びが現れる確率はどのくらいだろうか。つまり、**0から9の数字を等確率で767個並べたとき、6個以上の同じ数字の並びが現れる確率**をモンテカルロ法で概算してみよう。数字の並びは`999999`とは限らず、`000000`や`888888`でもよく、`0`から`9`のいずれかの連続した並びとする。サンプルサイズは`10 ** 5`ほどでよい（実行に30秒もかからないはずである）。結果とともに誤差の見積もりを表示せよ。その際、相対誤差は式(3)で評価するものとする。

<!-- textlint-enable -->

In [ ]:
# 課題解答8.3  <-- 提出する際に、この行を必ず含めること。

import random

## 脚注

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-1"></a>1.&nbsp;[^](#cite_ref-1)
この確率は$\frac{21632997006968743673674929893}{39614081257132168796771975168} \approx 0.546$である（解法は、たとえば[これ](http://zakii.la.coocan.jp/enumeration/52_cointoss.htm)）。

<!-- textlint-enable -->

<a name="cite_note-2"></a>2.&nbsp;[^](#cite_ref-2)
[`search`関数](https://docs.python.org/ja/3/library/re.html#re.search)は、指定されたパターンが見つかれば[マッチオブジェクト](https://docs.python.org/ja/3/library/re.html#match-objects)を返す。見つからなければ`None`を返す。ここでは`if`文の条件として与えているが、オブジェクトは`True`、`None`は`False`とみなされる（`bool`に変換される）ことを利用している。

<a name="cite_note-3"></a>3.&nbsp;[^](#cite_ref-3)
ある量の真値$x$とその近似値$\bar{x}$から、
絶対誤差$\varepsilon_\text{abs}$を、
$$
\varepsilon_\text{abs} = \lvert \bar{x} - x \rvert ,
$$
相対誤差$\varepsilon_\text{rel}$を、
$$
\varepsilon_\text{rel} = \left\lvert \frac{\varepsilon_\text{abs}}{x} \right\rvert \simeq \left\lvert \frac{\varepsilon_\text{abs}}{\bar{x}} \right\rvert,
$$
で定義する。

<!-- textlint-disable
ja-technical-writing/sentence-length
-->

今回のプログラムで表示されている「誤差」とは、正確には絶対誤差$\varepsilon_\text{abs}$に対する見積もりのことであり、シミュレーションで得られた近似値$\bar{x}$に、相対誤差$\varepsilon_\text{rel}$の見積もりを掛けることで計算している。

<!-- textlint-enable -->

<a name="cite_note-4"></a>4.&nbsp;[^](#cite_ref-4)
確率$p$で値が$1$、確率$(1-p)$で値が$0$となるような確率変数$X_i$を考える。

$$
\begin{cases}
  P(X_i=1) = p, \\
  P(X_i=0) = 1 - p.
\end{cases}
$$

<!-- textlint-disable ja-engineering-paper/prh,ja-technical-writing/sentence-length -->

いまの問題では、パターンがあるときを$X_i=1$、ないときを$X_i=0$に対応させればよい。この次にある、領域の面積（あるいは円周率）を求める問題でも同様に、領域に入ったか入ってないかをそれぞれ$1$と$0$に対応させる。$X_i$の期待値と分散はそれぞれ$E[X_i] = p$、$V[X_i] = p(1-p)$である。このような確率変数を$n$個足し、$n$で割って$\bar{X}$とする。
$$
\bar{X} = \frac{X_1 + X_2 + \dots + X_n}{n}.
$$
すると$E\left[\bar{X}\right] = p$、$V\left[\bar{X}\right] = \frac{p(1-p)}{n}$である。ただし、$i\ne j$に対し$X_i$と$X_j$は無相関（uncorrelated）であると仮定した。$n$が十分大きければ$\bar{X}$は正規分布にほぼ従う（中心極限定理）。このとき、$1\sigma$に対応する相対誤差は$\sqrt{V\left[\bar{X}\right]}\bigm/E\left[\bar{X}\right]=\sqrt{\frac{1-p}{np}}$である。$p\ge\frac{1}{2}$であれば$\sqrt{\frac{1-p}{np}}\le\sqrt{\frac{1}{n}}$となる。いまの問題では確かに$p$が$0.5$よりも大きいので（誤差の上限を求めることとして）比例係数を無視した。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-5"></a>5.&nbsp;[^](#cite_ref-5)
（統計誤差が入るモンテカルロ法による数値実験であれ、偶然誤差の入る現実の実験であれ）統計誤差とはあくまでも測定結果と真の値のずれの程度を統計的に示すもので、その誤差範囲の中に必ず真の値があると主張しているものでは**ない**ことに注意すること。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-6"></a>6.&nbsp;[^](#cite_ref-6)
この確率は$\frac{63922909553544897203855125999}{79228162514264337593543950336} \approx 0.807$である（解法は、たとえば[これ](https://egory-cat.hatenablog.com/entry/2018/02/19/034657)）。

<!-- textlint-enable -->

<a name="cite_note-7"></a>7.&nbsp;[^](#cite_ref-7)
たとえば（ランダムにするため）目をつぶった状態でダーツをしてみてはどうだろうか（正方形の中に入る回数が減りそうなので効率が悪そうだが）。

<!-- textlint-disable ja-technical-writing/max-ten,ja-engineering-paper/prh,jtf-style/4.3.7.山かっこ<>,ja-technical-writing/no-doubled-conjunctive-particle-ga -->

<a name="cite_note-8"></a>8.&nbsp;[^](#cite_ref-8)
等号成立（$x^2+y^2=1$）の場合は点が円周上にあることとなるが、これを内部に属するとしても、外部に属するとしても、結果には影響しない。実際上、確率が0だからである。同様に、正方形内部のランダムな点を発生させるときも、$0 \le x < 1$の代わりに、$0 \le x \le 1$ としようが、$0 < x < 1$ としようが、結果には影響しない。

<!-- textlint-enable -->

<a name="cite_note-9"></a>9.&nbsp;[^](#cite_ref-9)
なお、運が悪いと外れる（約6%の確率）。

<a name="cite_note-10"></a>10.&nbsp;[^](#cite_ref-10)
半径$r$の$n$次元超球の超体積$V_n(r)$はガンマ関数を用いて次のように書ける。

$$
V_n(r) = \frac{\pi^{\frac{n}{2}}}{\Gamma\left(\frac{n}{2}+1\right)} r^n.
$$

これは、正の整数$k$に対して次のように書き下せる。

<!-- textlint-disable -->

\begin{align}
V_{2k}(r) &= \frac{\pi^k}{k!} r^{2k}, \\
V_{2k+1}(r) &= \frac{2 (k!) (4\pi)^k }{(2k+1)!} r^{2k+1}.
\end{align}

<!-- textlint-enable -->

1次元、2次元、3次元に対し、それぞれ直径、円の面積、球の体積となることが分かるだろう。

<a name="cite_note-11"></a>11.&nbsp;[^](#cite_ref-11)
$n$次元超球の超体積は$n \approx 5.256$までは$n$が大きくなるにつれて増加するが、それ以降は単調に減少し、$n\to 0$で$0$に漸近する。つまり、ランダムに発生させた点がどんどん超球に入らなくなっていく。高次元空間では3次元空間に慣れ親しみ魂を縛られている人々の直感の斜め上を行くことが起きる（たとえば[これ](https://windfall.hatenablog.com/entry/2015/07/02/084623)）。

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-12"></a>12.&nbsp;[^](#cite_ref-12)
1次元の積分を数値的に実行したいのなら、この問題のようなモンテカルロ・シミュレーションよりも、ずっと高速で正確な方法がある。ガウス＝クロンロッド求積法による自動積分や、あるいは（この問題に対しては）単に台形公式やシンプソンの公式などをもとにした合成則でもよいだろう。また、通常モンテカルロ積分といえば、高次元の積分に対する重点的サンプリング（importance sampling）やマルコフ連鎖モンテカルロ法（Markov chain Monte Carlo）を用いたものを指す。

<!-- textlint-enable -->

<a name="cite_note-13"></a>13.&nbsp;[^](#cite_ref-13)
領域を表す不等式の不等号の向きを調べるには、$x=y=0$を代入すればよい（原点は領域の中にある）。

<a name="cite_note-14"></a>14.&nbsp;[^](#cite_ref-14)
陰関数表示に$y=0$、あるいは媒介変数表示に$t=0$を代入すると、$x$の上限は$x_\text{max}=a=4/\sqrt{6}\approx1.633$と求まる。
一方で、[`random`関数](https://docs.python.org/ja/3/library/random.html#random.random)をそのまま使うと0から1までの乱数が返ってくることに注意する。これに自分で変換を施すか、あるいは[`uniform`関数](https://docs.python.org/ja/3/library/random.html#random.uniform)を用いて、必要な範囲での乱数を発生させる必要がある。